In [2]:
import numpy as np
import pandas as pd
from utils import get_prices

In [3]:
asset_type="stock"
adjustment="adj_splitdiv"
period="full"
timeframe="1min"
ticker_symbol="RDFN"
prices_df = get_prices(asset_type, adjustment, period, timeframe, ticker_symbol)
prices_df

Looking for RDFN stock csv file in /media/reggie/reg_ext/frd-historical/data/stock/1min/csv/
RDFN_full_1min_adjsplitdiv.txt
RDFN_full_1min_adjsplitdiv.txt
/media/reggie/reg_ext/frd-historical/data/stock/1min/csv/RDFN_full_1min_adjsplitdiv.txt
(636618, 6)


,date,open,high,low,close,volume,day,time,time_id
0,2017-07-28 11:05:00,19.560,19.760,19.510,19.670,1573865,2017-07-28,11:05:00,666
1,2017-07-28 11:06:00,19.710,20.070,19.630,19.970,389751,2017-07-28,11:06:00,667
2,2017-07-28 11:07:00,19.980,20.360,19.920,20.080,218754,2017-07-28,11:07:00,668
3,2017-07-28 11:08:00,20.150,20.692,20.125,20.692,253592,2017-07-28,11:08:00,669
4,2017-07-28 11:09:00,20.680,20.920,20.220,20.400,125916,2017-07-28,11:09:00,670
...,...,...,...,...,...,...,...,...,...
636613,2024-03-08 19:18:00,7.170,7.180,7.170,7.180,3500,2024-03-08,19:18:00,1159
636614,2024-03-08 19:21:00,7.160,7.160,7.150,7.150,1200,2024-03-08,19:21:00,1162
636615,2024-03-08 19:22:00,7.170,7.170,7.170,7.170,100,2024-03-08,19:22:00,1163
636616,2024-03-08 19:29:00,7.179,7.179,7.179,7.179,500,2024-03-08,19:29:00,1170


In [19]:
prices_df.to_csv('temp-rdfn-prices.csv', index=False)

In [7]:
X = prices_df["close"].values
n = X.shape[0]
print(n)
X

636618


array([19.67 , 19.97 , 20.08 , ...,  7.17 ,  7.179,  7.18 ])

In [16]:
import pandas as pd

sequence_length = 10
sequences = []
targets = []
for i in range(n - sequence_length):
    sequences.append(X[i:i + sequence_length])
    targets.append(X[i + sequence_length])

sequences = np.array(sequences)
targets = np.array(targets)

df = pd.DataFrame(sequences)
df.columns = [f"x_{i}" for i in range(df.shape[1])]
df['y'] = targets
# rearrange column order
df = df[['y'] + [f"x_{i}" for i in range(df.shape[1] - 1)]]
df

,y,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9
0,19.315,19.6700,19.9700,20.0800,20.6920,20.4000,19.950,20.000,19.943,19.900,19.790
1,19.650,19.9700,20.0800,20.6920,20.4000,19.9500,20.000,19.943,19.900,19.790,19.315
2,19.880,20.0800,20.6920,20.4000,19.9500,20.0000,19.943,19.900,19.790,19.315,19.650
3,19.900,20.6920,20.4000,19.9500,20.0000,19.9430,19.900,19.790,19.315,19.650,19.880
4,19.800,20.4000,19.9500,20.0000,19.9430,19.9000,19.790,19.315,19.650,19.880,19.900
...,...,...,...,...,...,...,...,...,...,...,...
636603,7.180,7.1200,7.1200,7.1200,7.1500,7.1502,7.160,7.150,7.100,7.100,7.120
636604,7.150,7.1200,7.1200,7.1500,7.1502,7.1600,7.150,7.100,7.100,7.120,7.180
636605,7.170,7.1200,7.1500,7.1502,7.1600,7.1500,7.100,7.100,7.120,7.180,7.150
636606,7.179,7.1500,7.1502,7.1600,7.1500,7.1000,7.100,7.120,7.180,7.150,7.170


In [17]:
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size
        self.W_xh = np.random.randn(hidden_size, input_size)
        self.W_hh = np.random.randn(hidden_size, hidden_size)
        self.W_hy = np.random.randn(output_size, hidden_size)
        self.b_h = np.zeros((hidden_size, 1))
        self.b_y = np.zeros((output_size, 1))
    
    def forward(self, inputs):
        h_t = np.zeros((self.hidden_size, 1))
        for x_t in inputs:
            x_t = x_t.reshape(-1, 1)
            h_t = np.tanh(np.dot(self.W_xh, x_t) + np.dot(self.W_hh, h_t) + self.b_h)
        y_t = np.dot(self.W_hy, h_t) + self.b_y
        return y_t, h_t

    def compute_loss(self, predicted, actual):
        return np.square(predicted - actual).sum() / 2

    def backward(self, inputs, h_t, predicted, actual, learning_rate=0.001):
        dL_dy = predicted - actual.reshape(-1, 1)
        dL_dWhy = np.dot(dL_dy, h_t.T)
        dL_dby = dL_dy
        
        dL_dh = np.dot(self.W_hy.T, dL_dy) * (1 - h_t ** 2)
        dL_dWxh = np.dot(dL_dh, inputs[-1].reshape(1, -1))
        dL_dWhh = np.dot(dL_dh, h_t.T)
        dL_dbh = dL_dh
        
        self.W_xh -= learning_rate * dL_dWxh
        self.W_hh -= learning_rate * dL_dWhh
        self.W_hy -= learning_rate * dL_dWhy
        self.b_h -= learning_rate * dL_dbh
        self.b_y -= learning_rate * dL_dby

In [18]:
# Hyperparameters
input_size = 1
hidden_size = 10
output_size = 1
learning_rate = 0.001
epochs = 100

# Initialize the model
rnn = SimpleRNN(input_size, hidden_size, output_size)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(sequences)):
        inputs = sequences[i].reshape(sequence_length, 1)
        target = targets[i]
        
        predicted, h_t = rnn.forward(inputs)
        loss = rnn.compute_loss(predicted, target)
        total_loss += loss
        
        rnn.backward(inputs, h_t, predicted, target, learning_rate)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(sequences)}')


KeyboardInterrupt: 